# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

In [2]:
if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import Modules

In [3]:
# install required modules quietly
required_packages = ['geopandas', 'pyspark', 'azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

In [4]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import math
import json

import pandas as pd
from calendar import monthrange
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [5]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").config(
    "spark.jars.packages", 
    "org.apache.hadoop:hadoop-azure:3.3.1,com.microsoft.azure:azure-storage:8.6.6"
    ).getOrCreate()
# Check Spark Session Information
spark

# Define Constants

In [6]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'

In [7]:
# File
container = "baseline-data"
ext = "parquet"
ver = "0"
blob_name_base = f"baseline_all_v_{ver}"
train_blob_name_base = f"baseline-train-v-{ver}"
test_blob_name_base = f"baseline-test-v-{ver}"

In [8]:
# "Golden" Sites
tier1_sites = ["US-MMS", "US-Vcp", "FR-Pue", "CH-Lae", "US-Var", "US-Ne2", "ES-LJu", "US-Ton"]
tier2_sites = ["US-UMB", "US-Me2", "FI-Hyy", "US-NR1", "IT-Lav", "US-Wkg", "US-ARM", "US-SRM"]

train_sites = tier1_sites + tier2_sites

# Selected Test Sites
test_sites = ["US-GLE", # ENF, Cold
              "US-AR1", # GRA, Temperate
              "US-Seg", # GRA, Arid
              "US-FR2", # WSA, Temperate
              "ES-LM2", # WSA, Arid
              "CA-Cbo", # DBF, Cold
              "FR-Lam", # CRO, Temperate
              "IT-Cpz", # EBF, Temperate
              "CN-Cha", # MF Cold
              "IT-Lsn", # OSH, Temperate
              ]

In [9]:
# Define features and target variables of the data pipelines
included_features = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA',
                     'datetime', 'year', 'month', 'day', 'hour', 'date',
                     'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 
                     'IGBP', 'koppen']
target_variable_qc = 'NEE_VUT_REF_QC'
target_variable = 'GPP_NT_VUT_REF'

# Get Gold Sample Site Data

In [10]:
# Load site metadata
included_site_features = ['site_id', 'filename', 'elevation', 'lat', 'long',
                          'koppen_sub', 'koppen_main', 'koppen_name',
                          'c3c4', 'c4_percent']
site_metadata_df = pd.read_csv(site_metadata_filename, usecols = included_site_features)

# only focus on target sites
site_metadata_df = site_metadata_df.loc[site_metadata_df['site_id'].isin(train_sites + test_sites)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df.reset_index(inplace=True, drop=True)
site_metadata_df

size:(26, 10)


,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,c3c4,c4_percent,filename
0,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,C3,12.17000,data_full_half_hourly_raw_v0_1_CN-Cha.csv
1,FR-Pue,270.00000,43.74130,3.59570,8,3,Csa,C3,6.59000,data_full_half_hourly_raw_v0_1_FR-Pue.csv
2,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,C3,0.00000,data_full_half_hourly_raw_v0_1_IT-Cpz.csv
3,US-GLE,3197.00000,41.36653,-106.23990,27,4,Dfc,C3,0.16000,data_full_half_hourly_raw_v0_1_US-GLE.csv
4,US-NR1,3050.00000,40.03290,-105.54640,27,4,Dfc,C3,0.35000,data_full_half_hourly_raw_v0_1_US-NR1.csv
5,US-Ne2,362.00000,41.16487,-96.47010,25,4,Dfa,rotation,48.91000,NaN
6,US-SRM,1120.00000,31.82140,-110.86610,6,2,BSh,C3,55.39000,data_full_half_hourly_raw_v0_1_US-SRM.csv
7,US-Ton,177.00000,38.43160,-120.96598,8,3,Csa,C3,0.00000,data_full_half_hourly_raw_v0_1_US-Ton.csv
8,US-Var,129.00000,38.41330,-120.95070,8,3,Csa,C3,0.00000,data_full_half_hourly_raw_v0_1_US-Var.csv
9,US-Wkg,1531.00000,31.73650,-109.94190,7,2,BSk,C4,40.93000,data_full_half_hourly_raw_v0_1_US-Wkg.csv


# Stage 1: Trim and Merge Site Metadata

In [11]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']

In [12]:
# Initial data clean and feature selections from raw data
data_df = data_cleanup(raw_data_dir, site_metadata_df[['site_id','filename']],
                  target_variable, target_variable_qc,
                  included_features)
print(f"Data size after cleanup: {data_df.shape}")

# Merge with site metadata
data_df = merge_site_metadata(data_df, site_metadata_df.drop(['filename', 'koppen_main', 'koppen_name'], axis=1))
print(f"Data size after after merged with site metadata: {data_df.shape}")

# Drop rows with NA
check_and_drop_na(data_df)
print(f"Data size after after final drop: {data_df.shape}")

#reorder columns
features = data_df.columns.to_list()
features.remove(target_variable)
data_df = data_df[([target_variable] + features)]

display(data_df.head())

CN-Cha: (16228, 28)
FR-Pue: (117200, 28)
IT-Cpz: (59175, 28)
US-GLE: (54687, 28)
US-NR1: (98652, 28)

ERROR: US-Ne2 is mssing hourly data.
US-SRM: (95419, 28)
US-Ton: (113031, 28)
US-Var: (119950, 28)
US-Wkg: (93319, 28)
CA-Cbo: (79273, 28)
US-AR1: (28956, 28)
US-ARM: (125756, 28)
US-FR2: (30426, 28)

ERROR: US-MMS is mssing hourly data.
US-Me2: (99780, 28)
US-Seg: (91884, 28)
US-UMB: (70639, 28)
US-Vcp: (78491, 28)
CH-Lae: (112718, 28)
ES-LJu: (112724, 28)
ES-LM2: (58806, 28)
FI-Hyy: (127362, 28)
FR-Lam: (115812, 28)
IT-Lav: (120885, 28)
IT-Lsn: (40182, 28)
Data size after cleanup: (2061355, 28)
Data size after after merged with site metadata: (2061355, 34)
Data has NA.


,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,timestep_idx,elevation,lat,long,koppen_sub,c3c4,c4_percent
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16228,0,0,0,0,0


Data size after after final drop: (2045127, 34)


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,timestep_idx,elevation,lat,long,koppen_sub,c3c4,c4_percent
16228,-0.53574,5.31100,25.01600,272.21800,1.70800,0.00000,97.93900,2001-01-01 08:30:00,2001,1,1,8,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,7649,270.00000,43.74130,3.59570,8,C3,6.59000
16229,0.86438,5.74400,59.73400,272.21800,1.73800,0.00000,97.93900,2001-01-01 09:00:00,2001,1,1,9,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,7650,270.00000,43.74130,3.59570,8,C3,6.59000
16230,-0.02627,6.17600,91.23500,272.21800,1.76700,0.00000,97.93900,2001-01-01 09:30:00,2001,1,1,9,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,7651,270.00000,43.74130,3.59570,8,C3,6.59000
16231,-0.17229,6.60800,79.26400,333.93300,1.79700,0.05000,97.93900,2001-01-01 10:00:00,2001,1,1,10,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,7652,270.00000,43.74130,3.59570,8,C3,6.59000
16232,1.20865,7.04300,94.92900,333.93300,1.81700,0.00000,97.92300,2001-01-01 10:30:00,2001,1,1,10,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,7653,270.00000,43.74130,3.59570,8,C3,6.59000


In [13]:
data_df.describe()

,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,minute,timestep_idx,elevation,lat,long,koppen_sub,c4_percent
count,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000
mean,5.87229,15.18393,376.25804,319.49078,10.11018,0.03958,91.62118,2010.10650,6.55024,15.75130,11.92300,0.29796,0.52726,0.13143,0.08435,0.24830,0.04657,0.07608,0.26551,0.20770,0.12646,15.00078,127323.88200,901.20832,41.38344,-58.39359,15.86035,7.39929
std,7.61167,9.17084,267.25693,49.30580,9.61124,0.19043,9.25367,4.65177,3.00549,8.81239,3.75671,0.13328,0.22261,0.07261,0.06102,0.06284,0.04531,0.04748,0.06925,0.09472,0.08461,15.00000,81833.07019,906.36713,7.03857,57.53619,8.51506,14.29274
min,-49.73720,-30.68700,0.00100,107.63900,0.00000,0.00000,66.03100,2001.00000,1.00000,1.00000,3.00000,-0.11958,-0.18252,-0.01764,0.00540,0.03050,0.00000,0.00000,0.00000,0.01320,0.00000,0.00000,10.00000,1.00000,29.94950,-121.55740,5.00000,0.00000
25%,0.44478,8.97400,138.39100,285.18600,3.23000,0.00000,85.19600,2007.00000,4.00000,8.00000,9.00000,0.20223,0.33970,0.07913,0.03670,0.20250,0.02110,0.04440,0.20950,0.13000,0.05590,0.00000,59868.00000,180.00000,36.60580,-109.94190,8.00000,0.00000
50%,3.14854,15.31700,343.34100,321.12900,6.98900,0.00000,97.16200,2010.00000,7.00000,16.00000,12.00000,0.28931,0.55534,0.11931,0.06750,0.23890,0.03570,0.06640,0.27080,0.19530,0.10880,30.00000,119016.00000,314.00000,40.03290,-97.48880,14.00000,0.35000
75%,9.46059,21.72100,585.40800,354.95800,13.69700,0.00000,99.03900,2014.00000,9.00000,23.00000,15.00000,0.36828,0.70229,0.16489,0.11910,0.28780,0.05610,0.09460,0.32010,0.28260,0.18360,30.00000,183330.00000,1531.00000,44.45230,3.59570,26.00000,6.59000
max,85.03090,44.08700,1094.34100,473.08500,80.09100,15.49300,103.92100,2020.00000,12.00000,31.00000,23.00000,2.38835,0.93551,0.42385,0.81200,0.77620,0.76890,0.80530,0.46660,0.47920,0.43470,30.00000,408418.00000,3197.00000,61.84741,24.29477,27.00000,55.39000


In [14]:
data_df.site_id.unique()

array(['FR-Pue', 'IT-Cpz', 'US-GLE', 'US-NR1', 'US-SRM', 'US-Ton',
       'US-Var', 'US-Wkg', 'CA-Cbo', 'US-AR1', 'US-ARM', 'US-FR2',
       'US-Me2', 'US-Seg', 'US-UMB', 'US-Vcp', 'CH-Lae', 'ES-LJu',
       'ES-LM2', 'FI-Hyy', 'FR-Lam', 'IT-Lav', 'IT-Lsn'], dtype=object)

In [15]:
data_df.IGBP.unique()

array(['EBF', 'ENF', 'WSA', 'GRA', 'DBF', 'CRO', 'MF', 'OSH'],
      dtype=object)

In [16]:
data_df.koppen.unique()

array(['Temperate', 'Cold', 'Arid'], dtype=object)

# CheckPoint: Upload Data to Azure Storage Blob as Parquet

In [17]:
# Upload to Azure Storage Blob
# ref: https://stackoverflow.com/a/54666079
data_cleanup_checkpoint = True
tag = "raw"
blob_name = f"{blob_name_base}_{tag}.{ext}"

if data_cleanup_checkpoint:

  parquet_file = BytesIO()
  data_df.to_parquet(parquet_file, engine='pyarrow')
  parquet_file.seek(0)

  azStorageClient = AzStorageClient(az_cred_file)
  azStorageClient.uploadBlob(container, blob_name, parquet_file, overwrite=True)

File uploaded to baseline-data/baseline_all_v_0_raw.parquet


In [18]:
# Save no transformed data
get_non_transform_train_test = True
if get_non_transform_train_test:
  data_transformer = TFTDataTransformer(train_sites, test_sites,
                                        None, data_df)
  train_df, test_df = data_transformer.get_test_train_raw()
  print("Train data peak:")
  display(train_df.head(5))
  print("Test data peak:")
  display(test_df.head(5))

  train_blob_name= f"{train_blob_name_base}-{tag}.{ext}"
  test_blob_name= f"{test_blob_name_base}-{tag}.{ext}"
  data_transformer.upload_train_test_to_azure(az_cred_file, container,\
                                              train_blob_name, test_blob_name)

Data size: (2045127, 33).
Train data size: (1485926, 33).
Test data size: (559201, 33).
Train data peak:


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,datetime,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,timestep_idx,elevation,lat,long,koppen_sub,c3c4,c4_percent
16228,-0.53574,5.31100,25.01600,272.21800,1.70800,0.00000,97.93900,2001-01-01 08:30:00,2001,1,1,8,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,7649,270.00000,43.74130,3.59570,8,C3,6.59000
16229,0.86438,5.74400,59.73400,272.21800,1.73800,0.00000,97.93900,2001-01-01 09:00:00,2001,1,1,9,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,7650,270.00000,43.74130,3.59570,8,C3,6.59000
16230,-0.02627,6.17600,91.23500,272.21800,1.76700,0.00000,97.93900,2001-01-01 09:30:00,2001,1,1,9,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,7651,270.00000,43.74130,3.59570,8,C3,6.59000
16231,-0.17229,6.60800,79.26400,333.93300,1.79700,0.05000,97.93900,2001-01-01 10:00:00,2001,1,1,10,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,7652,270.00000,43.74130,3.59570,8,C3,6.59000
16232,1.20865,7.04300,94.92900,333.93300,1.81700,0.00000,97.92300,2001-01-01 10:30:00,2001,1,1,10,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,7653,270.00000,43.74130,3.59570,8,C3,6.59000


Test data peak:


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,datetime,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,timestep_idx,elevation,lat,long,koppen_sub,c3c4,c4_percent
133428,2.19090,4.66000,5.44900,218.92000,2.90400,0.00000,100.76500,2001-01-01 07:30:00,2001,1,1,7,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,13743,68.00000,41.70525,12.37611,8,C3,0.00000
133429,2.83647,5.08600,71.31800,218.92000,2.95200,0.00000,100.78900,2001-01-01 08:00:00,2001,1,1,8,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,0,IT-Cpz,13744,68.00000,41.70525,12.37611,8,C3,0.00000
133430,4.03906,5.51100,132.32000,218.92000,3.00000,0.00000,100.81300,2001-01-01 08:30:00,2001,1,1,8,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,13745,68.00000,41.70525,12.37611,8,C3,0.00000
133431,7.79772,5.93700,187.41100,218.92000,3.04800,0.00000,100.83700,2001-01-01 09:00:00,2001,1,1,9,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,0,IT-Cpz,13746,68.00000,41.70525,12.37611,8,C3,0.00000
133432,6.93281,6.36200,235.65000,218.92000,3.09600,0.00000,100.86100,2001-01-01 09:30:00,2001,1,1,9,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,13747,68.00000,41.70525,12.37611,8,C3,0.00000


Uploading train dataset to baseline-train-v-0-raw.parquet...
File uploaded to baseline-data/baseline-train-v-0-raw.parquet
Uploading test dataset to baseline-test-v-0-raw.parquet...
File uploaded to baseline-data/baseline-test-v-0-raw.parquet


# Stage 2: Data Transform - Convert to Model Ready Data
- Encode data
- Split into train and test
- Assemble data through VectorAssembler
- Normalized data through minmax

In [24]:
load_data_from_previous_checkpoint = True
useSpark = False

raw_data_file_path = None
if load_data_from_previous_checkpoint:
  data_df = None
  raw_data_file_path = tmp_dir + os.sep + blob_name
  print(f"loading {raw_data_file_path}...")
  if not (os.path.exists(raw_data_file_path)):
      if not (os.path.exists(tmp_dir)):
          os.mkdir(tmp_dir)
      azStorageClient = AzStorageClient(az_cred_file)
      file_stream = azStorageClient.downloadBlob2Stream(container, blob_name)
      data_df = pd.read_parquet(file_stream, engine='pyarrow')
      data_df.to_parquet(raw_data_file_path)
  
if useSpark:
  data_transformer = PySparkMLDataTransformer(spark, train_sites, test_sites,
                                              raw_data_file_path, data_df)
else:
  data_transformer = TFTDataTransformer(train_sites, test_sites,
                                              raw_data_file_path, data_df)

loading /content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling/.tmp/baseline_all_v_0_raw.parquet...
Data size: (2045127, 33).


In [25]:
timestamp_cols = ['datetime', 'timestep_idx']
target_col = 'GPP_NT_VUT_REF'

if useSpark: # Spark ML Data Transformer
  categorical_cols = ['IGBP', 'c3c4', 'koppen_sub', 'koppen'] 
  data_transformer.data_transform(categorical_cols, timestamp_col, target_col)

  print("Train data peak:")
  data_transformer.train_df.show(5, False)
  print("Test data peak:")
  data_transformer.test_df.show(5, False)

  train_blob_name= f"{train_blob_name_base}"
  test_blob_name= f"{test_blob_name_base}"

else: # TFT Data Transformer
  categorical_cols = ['IGBP', 'c3c4', 'koppen_sub', 'koppen', 'site_id', 
                      'year', 'month', 'day', 'hour', 'minute']
  realNum_cols = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 
                  'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7',
                  'elevation', 'lat', 'long', 'c4_percent']
  backup_cols = ['IGBP', 'koppen','site_id']
  data_transformer.data_transform(categorical_cols, realNum_cols, backup_cols,\
                                  timestamp_cols, target_col)

  print("Train data peak:")
  display(data_transformer.train_df.head(5))
  print("Test data peak:")
  display(data_transformer.test_df.head(5))

  train_blob_name= f"{train_blob_name_base}.{ext}"
  test_blob_name= f"{test_blob_name_base}.{ext}"

Data size: (2045127, 36).
Data size after encoding: (2045127, 36)


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,datetime,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,timestep_idx,elevation,lat,long,koppen_sub,c3c4,c4_percent,IGBP_name,koppen_name,site_id_name
16228,-0.53574,5.31100,25.01600,272.21800,1.70800,0.00000,97.93900,2001-01-01 08:30:00,0,0,0,5,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,2,2,1,6,7649,270.00000,43.74130,3.59570,3,0,6.59000,EBF,Temperate,FR-Pue
16229,0.86438,5.74400,59.73400,272.21800,1.73800,0.00000,97.93900,2001-01-01 09:00:00,0,0,0,6,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,2,2,0,6,7650,270.00000,43.74130,3.59570,3,0,6.59000,EBF,Temperate,FR-Pue
16230,-0.02627,6.17600,91.23500,272.21800,1.76700,0.00000,97.93900,2001-01-01 09:30:00,0,0,0,6,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,2,2,1,6,7651,270.00000,43.74130,3.59570,3,0,6.59000,EBF,Temperate,FR-Pue
16231,-0.17229,6.60800,79.26400,333.93300,1.79700,0.05000,97.93900,2001-01-01 10:00:00,0,0,0,7,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,2,2,0,6,7652,270.00000,43.74130,3.59570,3,0,6.59000,EBF,Temperate,FR-Pue
16232,1.20865,7.04300,94.92900,333.93300,1.81700,0.00000,97.92300,2001-01-01 10:30:00,0,0,0,7,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,2,2,1,6,7653,270.00000,43.74130,3.59570,3,0,6.59000,EBF,Temperate,FR-Pue


Features(30): ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 'year', 'month', 'day', 'hour', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'IGBP', 'koppen', 'minute', 'site_id', 'elevation', 'lat', 'long', 'koppen_sub', 'c3c4', 'c4_percent']
Train data size: (1485926, 36).
Test data size: (559201, 36).
Normalizinf features (20): ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'elevation', 'lat', 'long', 'c4_percent']


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


Train data size: (1485926, 36).
Test data size: (559201, 36).
Train data peak:


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,datetime,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,timestep_idx,elevation,lat,long,koppen_sub,c3c4,c4_percent,IGBP_name,koppen_name,site_id_name
16228,-0.53574,-1.05761,-1.31284,-0.92961,-0.87415,-0.20769,0.80741,2001-01-01 08:30:00,0,0,0,5,-0.36105,0.89066,-0.34629,-1.02430,-1.59120,-0.90822,-1.06704,-1.57138,-1.09584,-1.51739,2,2,1,6,7649,-0.80873,0.25557,1.11494,3,0,-0.13980,EBF,Temperate,FR-Pue
16229,0.86438,-1.00974,-1.18356,-0.92961,-0.87103,-0.20769,0.80741,2001-01-01 09:00:00,0,0,0,6,-0.36105,0.89066,-0.34629,-1.02430,-1.59120,-0.90822,-1.06704,-1.57138,-1.09584,-1.51739,2,2,0,6,7650,-0.80873,0.25557,1.11494,3,0,-0.13980,EBF,Temperate,FR-Pue
16230,-0.02627,-0.96198,-1.06625,-0.92961,-0.86802,-0.20769,0.80741,2001-01-01 09:30:00,0,0,0,6,-0.36105,0.89066,-0.34629,-1.02430,-1.59120,-0.90822,-1.06704,-1.57138,-1.09584,-1.51739,2,2,1,6,7651,-0.80873,0.25557,1.11494,3,0,-0.13980,EBF,Temperate,FR-Pue
16231,-0.17229,-0.91422,-1.11083,0.37601,-0.86490,0.06870,0.80741,2001-01-01 10:00:00,0,0,0,7,-0.36105,0.89066,-0.34629,-1.02430,-1.59120,-0.90822,-1.06704,-1.57138,-1.09584,-1.51739,2,2,0,6,7652,-0.80873,0.25557,1.11494,3,0,-0.13980,EBF,Temperate,FR-Pue
16232,1.20865,-0.86613,-1.05250,0.37601,-0.86282,-0.20769,0.80560,2001-01-01 10:30:00,0,0,0,7,-0.36105,0.89066,-0.34629,-1.02430,-1.59120,-0.90822,-1.06704,-1.57138,-1.09584,-1.51739,2,2,1,6,7653,-0.80873,0.25557,1.11494,3,0,-0.13980,EBF,Temperate,FR-Pue


Test data peak:


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,datetime,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,timestep_idx,elevation,lat,long,koppen_sub,c3c4,c4_percent,IGBP_name,koppen_name,site_id_name
133428,2.19090,-1.12958,-1.38571,-2.05717,-0.74980,-0.20769,1.12696,2001-01-01 07:30:00,0,0,0,4,0.04053,0.94390,-0.08565,-0.98459,-1.21818,-0.63690,-1.00935,-0.32717,-1.09584,-0.90172,2,2,1,7,13743,-1.04175,-0.00633,1.26338,3,0,-0.55180,EBF,Temperate,IT-Cpz
133429,2.83647,-1.08248,-1.14042,-2.05717,-0.74481,-0.20769,1.12967,2001-01-01 08:00:00,0,0,0,5,0.04053,0.94390,-0.08565,-0.98459,-1.21818,-0.63690,-1.00935,-0.32717,-1.09584,-0.90172,2,2,0,7,13744,-1.04175,-0.00633,1.26338,3,0,-0.55180,EBF,Temperate,IT-Cpz
133430,4.03906,-1.03550,-0.91326,-2.05717,-0.73982,-0.20769,1.13238,2001-01-01 08:30:00,0,0,0,5,0.04053,0.94390,-0.08565,-0.98459,-1.21818,-0.63690,-1.00935,-0.32717,-1.09584,-0.90172,2,2,1,7,13745,-1.04175,-0.00633,1.26338,3,0,-0.55180,EBF,Temperate,IT-Cpz
133431,7.79772,-0.98840,-0.70810,-2.05717,-0.73483,-0.20769,1.13510,2001-01-01 09:00:00,0,0,0,6,0.04053,0.94390,-0.08565,-0.98459,-1.21818,-0.63690,-1.00935,-0.32717,-1.09584,-0.90172,2,2,0,7,13746,-1.04175,-0.00633,1.26338,3,0,-0.55180,EBF,Temperate,IT-Cpz
133432,6.93281,-0.94141,-0.52847,-2.05717,-0.72984,-0.20769,1.13781,2001-01-01 09:30:00,0,0,0,6,0.04053,0.94390,-0.08565,-0.98459,-1.21818,-0.63690,-1.00935,-0.32717,-1.09584,-0.90172,2,2,1,7,13747,-1.04175,-0.00633,1.26338,3,0,-0.55180,EBF,Temperate,IT-Cpz


# Checkpoint: Upload train and test to Azure Blob Storage

In [26]:
final_checkpoint = True

if final_checkpoint:
  data_transformer.upload_train_test_to_azure(az_cred_file, container, \
                                            train_blob_name, test_blob_name)

Uploading train dataset to baseline-train-v-0.parquet...
File uploaded to baseline-data/baseline-train-v-0.parquet
Uploading test dataset to baseline-test-v-0.parquet...
File uploaded to baseline-data/baseline-test-v-0.parquet
